<a href="https://colab.research.google.com/github/milmor/deep-puma/blob/main/Image-preprocessing/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Acceso a drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Bibliotecas 

In [2]:
!sudo apt install tesseract-ocr
!pip3 install pytesseract
!cp drive/MyDrive/HackathonRIIAA2021/Data/spa.traineddata /usr/share/tesseract-ocr/4.00/tessdata/

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,813 kB/s)
debconf: unable to initi

In [3]:
import os 
import pytesseract as ocr
import cv2 as cv
import numpy as np
from tqdm import tqdm

### Función para procesar imagenes y utilizar Tessearct-OCR

In [4]:
def get_processed_images(src_path, processed_path, texts_path, bin_option=True):
  source = [f for f in os.listdir(src_path) if os.path.isfile(os.path.join(src_path, f))]
  print('\n[INFO] Se encontraron {} imagenes para procesar.\n'.format(len(source)))
  for index in tqdm(range(len(source))):
    # Lectura y escalado de imagenes #
    image_name = source[index].split('.')[0]
    image = cv.imread('{}{}'.format(src_path,source[index]))
    image = cv.resize(image, None, fx=1.3, fy=1.3, interpolation=cv.INTER_CUBIC)
    """
    Preprocesamiento de imagenes
    """
    # Eliminación de ruido causado por sombras en las imagenes #
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    dilated_image = cv.dilate(gray_image, np.ones((7,7), np.uint8))
    blur_image = cv.medianBlur(dilated_image,21)
    diff_image = 255 - cv.absdiff(gray_image, blur_image)
    norm_image = diff_image.copy()
    cv.normalize(diff_image, norm_image, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8UC1)
    thr_img = cv.threshold(norm_image, 230, 0, cv.THRESH_TRUNC)[1]
    cv.normalize(thr_img, thr_img, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8UC1)
    bin_image = cv.threshold(thr_img, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)[1]
    
    # detección de texto #
    bin_image_inv = cv.threshold(thr_img, 0, 255, cv.THRESH_OTSU | cv.THRESH_BINARY_INV)[1]
    rect_kernel = cv.getStructuringElement(cv.MORPH_RECT, (90, 90))
    dilation = cv.dilate(bin_image_inv, rect_kernel, iterations = 1)
    
    contours, hierarchy = cv.findContours(dilation, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    if bin_option:
        image_copy = bin_image.copy()
    else:
        image_copy = gray_image.copy()
    count = -1
    for cnt in contours:
        count +=1
        x,y,w,h = cv.boundingRect(cnt)
        cropped = image_copy[y:y + h, x:x + w]
        h,w = cropped.shape
        if h>490 and w>490:
            text = ocr.image_to_string(cropped, lang='spa')
            if len(text) > 0:
                cv.imwrite('{}{}_processed_{}.png'.format(processed_path,image_name,count), cropped)
                textfile = open('{}{}_text.txt'.format(texts_path,image_name),'w')
                textfile.write(text)
                textfile.close()

In [5]:
IMAGES_PATH = 'drive/MyDrive/Datos - Hackathon JusticIA/Fichas_auto/'
PROCESSED_PATH = 'drive/MyDrive/HackathonRIIAA2021/Processed_images/Fichas_auto/'
TEXTS_PATH = 'drive/MyDrive/HackathonRIIAA2021/Texts/Fichas_auto/'

In [6]:
get_processed_images(src_path=IMAGES_PATH, processed_path=PROCESSED_PATH, texts_path=TEXTS_PATH)


[INFO] Se encontraron 1000 imagenes para procesar.



  0%|          | 1/1000 [00:23<6:32:05, 23.55s/it]


TesseractError: ignored